In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [2]:
train_data = pd.read_csv('Train_Kaggle.csv')
macro_economics_data = pd.read_excel('MacroEconomicData.xlsx')

In [3]:
events_data = pd.read_excel('Events_HolidaysData.xlsx')
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
Year           150 non-null int64
MonthDate      150 non-null datetime64[ns]
Event          150 non-null object
DayCategory    150 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 4.8+ KB


In [4]:
all_weather_data = pd.DataFrame()
for i in ['2009','2010','2011','2012','2013','2014','2015','2016']:
    data = pd.read_excel('WeatherData.xlsx',sheet_name=i,
                         names=['Year', 'Month', 'Day', 'Temp_high', 'Temp_avg',
       'Temp_low', 'Dew_Point_high', 'Dew_Point_avg',
       'Dew_Point_low', 'Humidity_high', 'Humidity_avg',
       'Humidity_low', 'Sea_Level_Press_high',
       'Sea_Level_Press_avg', 'Sea_Level_Press_low',
       'Visibility_high', 'Visibility_avg', 'Visibility_low',
       'Wind_low', 'Wind_avg', 'Wind_high',
       'Precip_sum', 'WeatherEvent'])
    data['Year'] = int(i)
    all_weather_data = pd.concat([all_weather_data,data],axis=0)
    #print(all_weather_data.head())

In [5]:
all_weather_data.head()

,Year,Month,Day,Temp_high,Temp_avg,Temp_low,Dew_Point_high,Dew_Point_avg,Dew_Point_low,Humidity_high,Humidity_avg,Humidity_low,Sea_Level_Press_high,Sea_Level_Press_avg,Sea_Level_Press_low,Visibility_high,Visibility_avg,Visibility_low,Wind_low,Wind_avg,Wind_high,Precip_sum,WeatherEvent
0,2009,Jan,1.0,-3,-6,-9,-16,-17,-19,54,43,32,1025,1023,1015,16,16,16,37,18,60,0,NaN
1,2009,Jan,2.0,1,-2,-5,-3,-7,-17,78,57,36,1022,1012,1007,16,13,2,27,10,48,T,Snow
2,2009,Jan,3.0,3,1,-2,-5,-9,-13,72,54,35,1018,1015,1008,16,16,16,27,16,42,T,NaN
3,2009,Jan,4.0,6,1,-4,-10,-12,-13,55,42,29,1020,1017,1015,16,16,16,32,12,40,0,NaN
4,2009,Jan,5.0,6,5,3,-1,-5,-16,62,48,33,1016,1014,1013,16,16,16,23,11,34,T,NaN


In [6]:
all_weather_data[['Temp_high', 'Temp_avg', 'Temp_low',
       'Dew_Point_high', 'Dew_Point_avg', 'Dew_Point_low', 'Humidity_high',
       'Humidity_avg', 'Humidity_low', 'Sea_Level_Press_high',
       'Sea_Level_Press_avg', 'Sea_Level_Press_low', 'Visibility_high',
       'Visibility_avg', 'Visibility_low', 'Wind_low', 'Wind_avg', 'Wind_high',
       'Precip_sum']] = all_weather_data[['Temp_high', 'Temp_avg', 'Temp_low',
       'Dew_Point_high', 'Dew_Point_avg', 'Dew_Point_low', 'Humidity_high',
       'Humidity_avg', 'Humidity_low', 'Sea_Level_Press_high',
       'Sea_Level_Press_avg', 'Sea_Level_Press_low', 'Visibility_high',
       'Visibility_avg', 'Visibility_low', 'Wind_low', 'Wind_avg', 'Wind_high',
       'Precip_sum']].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [7]:
# missing data
# all_weather_data[all_weather_data.isnull().any(axis=1)]

# Prepare Weather Data

In [8]:
grouped_data = all_weather_data.groupby(['Year','Month'],as_index=False).mean()
grouped_data['Month'] = grouped_data['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,
                              'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})
grouped_data.sort_values(by=['Year','Month'],axis=0)
weather_grouped_data = grouped_data.sort_values(by=['Year','Month'],axis=0)
weather_grouped_data = weather_grouped_data[weather_grouped_data['Year'] != 2015]
weather_grouped_data = weather_grouped_data[weather_grouped_data['Year'] != 2016]
weather_grouped_data.head()

,Year,Month,Day,Temp_high,Temp_avg,Temp_low,Dew_Point_high,Dew_Point_avg,Dew_Point_low,Humidity_high,Humidity_avg,Humidity_low,Sea_Level_Press_high,Sea_Level_Press_avg,Sea_Level_Press_low,Visibility_high,Visibility_avg,Visibility_low,Wind_low,Wind_avg,Wind_high,Precip_sum
4,2009,1,16.0,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.000000,14.000000,11.451613,27.000000,12.000000,39.322581,2.803704
3,2009,2,14.5,6.964286,2.785714,-1.642857,-2.464286,-6.392857,-10.571429,69.535714,54.000000,38.107143,1021.714286,1017.071429,1012.464286,16.000000,15.178571,13.285714,27.928571,12.214286,41.642857,0.840385
7,2009,3,16.0,9.838710,5.967742,1.774194,0.258065,-2.903226,-6.741935,74.096774,57.064516,39.548387,1025.096774,1021.064516,1017.032258,16.000000,14.258065,10.677419,27.000000,11.387097,38.612903,1.481667
0,2009,4,15.5,17.100000,12.733333,7.833333,6.233333,2.533333,-1.200000,75.466667,55.700000,35.733333,1018.833333,1014.533333,1010.433333,15.900000,14.066667,10.866667,27.300000,11.366667,39.566667,4.411852
8,2009,5,16.0,21.354839,17.000000,12.483871,12.967742,10.096774,6.516129,85.645161,68.516129,50.935484,1020.419355,1017.161290,1014.129032,15.903226,12.451613,9.000000,22.379310,8.000000,32.620690,4.527586


In [9]:
weather_grouped_data.shape

(72, 22)

# Prepare Macro Economics Data

In [10]:
macro_economics_data = pd.read_excel('MacroEconomicData.xlsx')
macro_economics_data['Month'] = macro_economics_data['Year-Month'].apply(lambda x : x.split('-')[1].strip())
macro_economics_data['Year'] = macro_economics_data['Year-Month'].apply(lambda x : x.split('-')[0].strip())
macro_economics_data['Month'] = macro_economics_data['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,
                              'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})
macro_economics_data = macro_economics_data.drop('Year-Month',axis=1)
macro_economics_data = macro_economics_data[macro_economics_data['Year'] != '2015']
macro_economics_data = macro_economics_data[macro_economics_data['Year'] != '2016']
macro_economics_data['Year'] = macro_economics_data['Year'].astype('int64')
macro_economics_data.head()
macro_economics_data.shape

(72, 19)

# Split training data into three

In [11]:
train_women_clothing = train_data[train_data['ProductCategory'] == 'WomenClothing']
train_other_clothing = train_data[train_data['ProductCategory'] == 'OtherClothing']
train_men_clothing = train_data[train_data['ProductCategory'] == 'MenClothing']

# Split Events data into Federal & Event Holidays with count, then do a left join with sales data

In [12]:
# for later
events_data = pd.read_excel('Events_HolidaysData.xlsx')
events_data['Month'] = events_data['MonthDate'].apply(lambda x: str(x).split('-')[1])
events_data = events_data.drop('MonthDate',axis=1)
events_data['Month'] = events_data['Month'].replace({'01':1,'02':2,'03':3,'04':4,'05':5,'06':6,'07':7,'08':8,'09':9})
events_data['Month'] = events_data['Month'].astype('int64')
events_data = events_data[events_data['Year'] != 2015]
events_data = events_data[events_data['Year'] != 2016]
events_data_federal_holidays = events_data[events_data['DayCategory'] == 'Federal Holiday']
events_data_event_holidays = events_data[events_data['DayCategory'] == 'Event']

In [13]:
events_grouped = events_data_event_holidays.groupby(['Year','Month'],as_index=False).count()
federal_grouped = events_data_federal_holidays.groupby(['Year','Month'],as_index=False).count()

# Need to combine weather data too 

In [14]:
#combined_data_1 = pd.merge(macro_economics_data,events_data_event_holidays,how='left',left_on=['Month','Year'],
#                        right_on=['Month','Year'])
#combined_data_2 = pd.merge(combined_data_1,events_data_federal_holidays,how='left',left_on=['Month','Year'],
#                        right_on=['Month','Year'])
combined_data_1 = pd.merge(macro_economics_data,events_grouped,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])
combined_data_2 = pd.merge(combined_data_1,federal_grouped,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])

# Combine with separate train data from men , women and other clothing sales 

In [15]:
women_clothing_total_data = pd.merge(combined_data_2,train_women_clothing,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])
men_clothing_total_data = pd.merge(combined_data_2,train_men_clothing,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])
other_clothing_total_data = pd.merge(combined_data_2,train_other_clothing,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])

In [16]:
women_clothing_total_data.columns

Index(['Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI', 'PartyInPower',
       'unemployment rate', 'CommercialBankInterestRateonCreditCardPlans',
       'Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan',
       'Earnings or wages  in dollars per hour',
       'AdvertisingExpenses (in Thousand Dollars)',
       'Cotton Monthly Price - US cents per Pound(lbs)', 'Change(in%)',
       'Average upland planted(million acres)',
       'Average upland harvested(million acres)', 'yieldperharvested acre',
       'Production (in  480-lb netweright in million bales)',
       'Mill use  (in  480-lb netweright in million bales)', 'Exports',
       'Month', 'Year', 'Event_x', 'DayCategory_x', 'Event_y', 'DayCategory_y',
       'ProductCategory', 'Sales(In ThousandDollars)'],
      dtype='object')

In [23]:
women_clothing_total_data = pd.merge(combined_data_2,train_women_clothing,how='left',left_on=['Month','Year'],
                        right_on=['Month','Year'])
women_clothing_total_data = women_clothing_total_data.drop(['AdvertisingExpenses (in Thousand Dollars)','DayCategory_x','DayCategory_y'],axis=1)
women_clothing_total_data[['Event_x','Event_y']] = women_clothing_total_data[['Event_x','Event_y']].fillna(0)
women_clothing_total_data = women_clothing_total_data.dropna()
X = women_clothing_total_data[['Monthly Real GDP Index (inMillion$)','CPI','Month','Event_x']]
y = women_clothing_total_data['Sales(In ThousandDollars)']

In [27]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.7942396699280857

In [43]:
from sklearn.tree import DecisionTreeRegressor

In [40]:
dt_rg = DecisionTreeRegressor()

In [41]:
dt_rg.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [57]:
women_clothing_total_data.to_excel("women.xlsx")

In [44]:
from sklearn.ensemble import AdaBoostRegressor

In [45]:
ada_rg = AdaBoostRegressor()


In [46]:
ada_rg.fit(X,y)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [47]:
ada_rg.score(X,y)

0.9285800022025923